In [1]:
import pandas as pd
from io import StringIO
from typing import Optional, List

class DataLoader:
    """
    Parses European wholesale electricity price data, allowing filtering
    by country and date range.
    """
    def __init__(self, file_path="./data/european_wholesale_electricity_price_data_daily.csv"):
        """
        Initializes the parser and loads the data.

        Args:
            file_path (str): The path to the CSV file.
        """
        self.file_path = file_path
        self.data = self._load_data()

    def _load_data(self):
        """Loads and preprocesses the data from the CSV file."""
        try:
            df = pd.read_csv(self.file_path)
            # Convert 'Date' column to datetime objects
            df['Date'] = pd.to_datetime(df['Date'])
            # Drop ISO3 Code column
            df.drop(columns={'ISO3 Code'}, inplace=True)
            # Rename price column for easier access
            df.rename(columns={'Price (EUR/MWhe)': 'Price'}, inplace=True)
            print(f"Data loaded successfully from {self.file_path}")
            return df
        except FileNotFoundError:
            print(f"Error: File not found at {self.file_path}")
            return None
        except KeyError as e:
            print(f"Error: Expected column '{e}' not found in the CSV.")
            return None
        except Exception as e:
            print(f"Error loading or processing file: {e}")
            return None

    def get_data_by_country_and_range(self, time_range:str, country=None):
        """
        Filters the data for a specific country and time range.

        Args:
            country (str): The name of the country to filter by (e.g., 'Germany').
            time_range (str): A string representing the date range in the format
                              'YYYY-MM-DD,YYYY-MM-DD'.

        Returns:
            pandas.DataFrame: A DataFrame containing the filtered data,
                              or None if an error occurs or no data is found.
        """
        if self.data is None:
            print("Error: Data not loaded.")
            return None

        try:
            start_date_str, end_date_str = time_range.split(',')
            start_date = pd.to_datetime(start_date_str.strip())
            end_date = pd.to_datetime(end_date_str.strip())
        except ValueError:
            print("Error: Invalid time_range format. Please use 'YYYY-MM-DD,YYYY-MM-DD'.")
            return None
        except Exception as e:
             print(f"Error parsing time range: {e}")
             return None

        outputData = self.data.copy()
        # If a country is specified, filter the data by country, if not use all data
        if country is not None:
            outputData = self.data[self.data['Country'].str.lower() == country.lower()]

        # Filter by date range (inclusive)
        filtered_data = outputData[
            (outputData['Date'] >= start_date) & (outputData['Date'] <= end_date)
        ]

        if filtered_data.empty:
            print(f"Warning: No data found for country '{country}' within the range {time_range}.")
            return pd.DataFrame() # Return empty DataFrame

        return filtered_data.copy() # Return a copy to avoid SettingWithCopyWarning

    def get_all_data(self):
        """
        Returns the entire dataset.

        Returns:
            pandas.DataFrame: The entire dataset.
        """
        if self.data is None:
            print("Error: Data not loaded.")
            return None
        return self.data.copy()

    def get_country_list(self):
        """
        Returns a list of unique countries in the dataset.

        Returns:
            list: A list of unique country names.
        """
        if self.data is None:
            print("Error: Data not loaded.")
            return None
        return self.data['Country'].unique().tolist()
    
    def get_price_matrix(
        self,
        time_range: str,
        countries: List[str],
        fill_method: Optional[str] = None
    ) -> pd.DataFrame:
        """
        Returns a price matrix where:
        - Rows = dates
        - Columns = countries
        - Values = daily electricity prices

        Parameters:
        - time_range (str): e.g. "2021-05-10,2021-05-16"
        - countries (List[str]): list of country names to include
        - fill_method (Optional[str]): 'ffill', 'bfill', or None

        Returns:
        - pd.DataFrame: index=date, columns=country names, values=prices
        """
        start_date, end_date = time_range.split(",")

        # Filter the master data once
        df = self.data.copy()
        df = df[df["Country"].isin(countries)]
        df = df[(df["Date"] >= start_date) & (df["Date"] <= end_date)]

        # Pivot: index=date, columns=country, values=price
        price_matrix = df.pivot(index="Date", columns="Country", values="Price").sort_index()

        # Handle missing data
        if fill_method:
            price_matrix = price_matrix.fillna(method=fill_method)
        else:
            price_matrix = price_matrix.dropna()

        return price_matrix

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import yfinance as yf
import matplotlib.pyplot as plt

class CointegrationResidualGenerator:
    def __init__(self, price_data: pd.DataFrame, risk_free_rate_annual: float = 0.01):
        self.price_data = price_data
        self.risk_free_rate_daily = risk_free_rate_annual / 252
        self.returns = self._compute_excess_returns(price_data)
        self.cumulative_returns = self._compute_cumulative_returns(self.returns)
        self.asset_residuals = pd.DataFrame(index=self.cumulative_returns.index)
        self.betas = {}  # Store betas for each asset

    def _compute_excess_returns(self, price_data: pd.DataFrame) -> pd.DataFrame:
        """Computes daily excess log returns."""
        log_returns = np.log(price_data / price_data.shift(1)).dropna()
        excess_returns = log_returns.subtract(self.risk_free_rate_daily)
        return excess_returns

    def _compute_cumulative_returns(self, returns: pd.DataFrame) -> pd.DataFrame:
        """Computes cumulative returns."""
        cumulative_returns = returns.cumsum()
        return cumulative_returns

    def compute_all_asset_residuals(self):
        """Computes residuals for each asset treated as dependent variable."""
        for target_asset in self.cumulative_returns.columns:
            # Set current asset as dependent variable
            y = self.cumulative_returns[target_asset].values.reshape(-1, 1)
            X = self.cumulative_returns.drop(columns=[target_asset]).values
            X_cols = self.cumulative_returns.drop(columns=[target_asset]).columns

            # Fit linear regression: y ~ X
            model = LinearRegression().fit(X, y)
            betas = model.coef_[0]
            intercept = model.intercept_[0]

            # Predict values and compute residuals for this asset
            y_pred = model.predict(X).flatten()
            residuals = y.flatten() - y_pred

            # Store residuals in DataFrame
            self.asset_residuals[target_asset] = residuals
            # Store betas for this asset
            beta_series = pd.Series(betas, index=X_cols)
            beta_series['Intercept'] = intercept
            self.betas[target_asset] = beta_series

    def get_asset_residuals(self) -> pd.DataFrame:
        """Returns residuals for all assets."""
        if self.asset_residuals.empty:
            raise ValueError("Asset residuals not yet computed.")
        return self.asset_residuals

    def get_betas_for_asset(self, asset: str) -> pd.Series:
        """Returns betas used to form residuals for a specific asset."""
        if asset not in self.betas:
            raise ValueError(f"Betas for asset '{asset}' not found. Compute residuals first.")
        return self.betas[asset]

    def prepare_cnn_input_from_residuals(self, window: int = 30):
        """
        Prepares CNN input data by creating rolling cumulative residuals.
        
        Returns:
        - 3D numpy array: [samples, window, assets]
        """
        if self.asset_residuals.empty:
            raise ValueError("Asset residuals not yet computed.")

        cnn_input_list = []

        for start_idx in range(len(self.asset_residuals) - window + 1):
            # Slice window of residuals
            window_residuals = self.asset_residuals.iloc[start_idx:start_idx + window]
            # Cumulative sum within the window
            cumulative_window = window_residuals.cumsum()
            # Store the result
            cnn_input_list.append(cumulative_window.values)

        # Convert to 3D numpy array: [samples, window, assets]
        cnn_input_array = np.array(cnn_input_list)
        return cnn_input_array


In [20]:
import torch
import torch.nn as nn


class CNN(nn.Module):
    def __init__(self, input_length, num_features, num_filters= 8, num_classes=2, filter_size=2):
        """
        Initialize the CNN model based on the equations in the paper.
        
        Args:
            input_length (int): Length of the input sequence (L in the equations)
            num_features (int): Number of features per time step
            num_classes (int): Number of output classes
        """
        super(CNN, self).__init__()
        
        self.num_filters = num_filters  # Number of filters (D = 8 according to equation)
        self.filter_size = filter_size  # Filter size (filter_size = 2 according to equation)
        
        # First convolutional layer (Equation 3)
        # Input shape: [batch_size, num_features, input_length]
        self.conv1 = nn.Conv1d(
            in_channels=num_features,
            out_channels=self.num_filters,
            kernel_size=self.filter_size,
            stride=1,
            padding=0
        )
        
        # Second convolutional layer (Equation 4)
        self.conv2 = nn.Conv1d(
            in_channels=self.num_filters,
            out_channels=self.num_filters,
            kernel_size=self.filter_size,
            stride=1,
            padding=0
        )
        
        # Calculate output sizes after convolutions
        L_after_conv1 = input_length - self.filter_size + 1
        L_after_conv2 = L_after_conv1 - self.filter_size + 1
        
        # Fully connected layers
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(self.num_filters * L_after_conv2, num_classes)
        
        # Activation
        self.relu = nn.ReLU()
        
    def forward(self, x):
        """
        Forward pass through the network.
        
        Args:
            x (torch.Tensor): Input tensor of shape [batch_size, num_features, input_length]
                             Represents the x_i(t) vector in Equation 2
        
        Returns:
            torch.Tensor: Output predictions
        """
        # Store original input for skip connection (Equation 5)
        x_original = x
        
        # First convolutional layer (Equation 3)
        x = self.relu(self.conv1(x))
        
        # Second convolutional layer (Equation 4)
        x = self.relu(self.conv2(x))
        
        # Skip connection (Equation 5)
        # Need to adjust dimensions for skip connection
        # Cut or pad x_original to match x dimensions
        if x_original.shape[2] > x.shape[2]:
            # If original is longer, cut it
            diff = x_original.shape[2] - x.shape[2]
            x_skip = x_original[:, :, diff:]
        else:
            # If original is shorter, this would require padding
            # For simplicity, we'll just use the original shape
            x_skip = x_original
        
        # Apply the skip connection if dimensions match
        if x.shape == x_skip.shape:
            x = x + x_skip
        
        # Flatten and pass through fully connected layer
        x = self.flatten(x)
        x = self.fc(x)
        
        return x
    
    def get_parameters(self):
        return self.parameters()


In [21]:
import torch
import torch.nn as nn # building blocks for neural networks
import torch.nn.functional as F # access to functions like ReLU, sigmoid, etc.

class FNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=32):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, 1)

    # IDEA: maybe try other relu variants such as LeakyReLU, ELU, etc.
    def forward(self, x):
        # x shape: [batch_size, input_dim]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        w_eps = self.out(x) # equation (10), single scalar value for each input
        return w_eps
    
# equation (11) from project framework
def soft_normalize(weights):
    """
    Normalize allocation weights using L1 norm (sum of absolute values).
    weights: Tensor of shape [batch_size, 1]
    Returns: Normalized weights of shape [batch_size, 1]
    """
    l1_norm = torch.sum(torch.abs(weights), dim=0, keepdim=True) + 1e-8 # avoid division by zero
    normalized_weights = weights / l1_norm
    return normalized_weights

In [ ]:

# MAIN FUNCTION
### -------------- PARSING DATA -------------- ###
parser = DataLoader()

# Get list of countries
print("\n--- List of Countries ---")
all_countries_list = parser.get_country_list()
print(all_countries_list)

# Get daily price matrix for all countries for the entire year 2021
price_matrix = parser.get_price_matrix(
    time_range="2021-01-01,2021-12-31",
    countries=all_countries_list,
    fill_method="ffill"
)

# Get the raw daily returns for the price matrix
returns = price_matrix.pct_change().dropna()

# # Plot the price vs time, and the daily returns for the first country in the list, enable grid
# plt.figure(figsize=(12, 6))
# plt.subplot(2, 1, 1)
# plt.plot(price_matrix.index, price_matrix.iloc[:, 0], label=price_matrix.columns[0])
# plt.title("Price vs Time")
# plt.xlabel("Date")
# plt.ylabel("Price")
# plt.grid(True)
# plt.legend()
# plt.subplot(2, 1, 2)
# plt.plot(returns.index, returns.iloc[:, 0], label=returns.columns[0])
# plt.title("Daily Returns vs Time")
# plt.xlabel("Date")
# plt.ylabel("Daily Returns")
# plt.grid(True)
# plt.legend()
# plt.tight_layout()
# plt.show()


### -------------- COINTEGRATION RESIDUALS -------------- ###
# Create an instance of the CointegrationResidualGenerator
residual_generator = CointegrationResidualGenerator(price_matrix)

residual_generator.compute_all_asset_residuals()

# Get residuals
asset_residuals = residual_generator.get_asset_residuals()

# Get the input for CNN
# cnn_input contains a set of 329 data samples, each sample represents 30-day cumulative residuals for the 31 countries
cumulative_residual_window = 30
cnn_input = residual_generator.prepare_cnn_input_from_residuals(window=cumulative_residual_window)

# Get the next-day returns corresponding to each of the 30-day cumulative residuals as a DataFrame
next_day_returns = returns.iloc[cumulative_residual_window + 1:]

# Get the start index of the first 30-day cumulative residuals in the returns DataFrame
start_idx_in_returns = returns.index.get_loc(asset_residuals.index[0])
num_samples = len(asset_residuals) - cumulative_residual_window + 1
next_day_indices = [start_idx_in_returns + i + cumulative_residual_window for i in range(num_samples)]
next_day_returns = returns.iloc[next_day_indices]


### -------------- FEED THE 30-DAY CUMULATIVE RESIDUALS OF EVERY COUNTRY TO CNN+FNN -------------- ###
# Transform cnn_input to be compatible with the CNN input shape
cnn_input_array = cnn_input.transpose(0, 2, 1) # [samples, features, window]

# FILIP'S SECTION: CNN+FNN
# Hey Filip, this is the section where you can add your code to train the CNN+FNN model.
# cnn_input_array essentially contains 329 training data points, each data point is 30-day cumulative residuals for the 31 countries.
# So, for one "data point" you would feed the set of 30-day cumulative residuals for every country to the CNN+FNN model.
# One data point should result in a set of weights for each of the 31 countries.
# Each set of weights is used to calculate one next-day portfolio return.
# Repeat this for all 329 data points to get a set of 329 portfolio returns.
# You can then use these portfolio returns to calculate the Sharpe ratio
# Optimize the CNN+FFN to maximize the Sharpe ratio.



### -------------- GET THE WEIGHTS OUTPUTTED FROM CNN+FNN -------------- ###
# Get the weight outputted from the CNN+FNN model
# One weight for each country
weights = [] # Change this to the actual weights outputted from the CNN+FNN model

# Convert weights to  


# next_day_returns = np.array([0.01, 0.02, -0.01])
# normalized_weights = evaluator.normalize_weights_l1(raw_weights)
# next_day_portfolio_return = evaluator.compute_portfolio_return(raw_weights, next_day_returns)
# # Store the next day portfolio return
# evaluator.add_return(next_day_portfolio_return)
# print("Next Day Portfolio Return:", next_day_portfolio_return)


Data loaded successfully from ./data/european_wholesale_electricity_price_data_daily.csv

--- List of Countries ---
['Austria', 'Belgium', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom', 'Bulgaria', 'Serbia', 'Croatia', 'Montenegro', 'North Macedonia', 'Ireland']


C:\Users\Volter\AppData\Local\Temp\ipykernel_15748\797593302.py:142: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  price_matrix = price_matrix.fillna(method=fill_method)
c:\Users\Volter\anaconda3\envs\DeepLearningSTAT7007\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)
